In [1]:
import pandas as pd
import pyrankvote
from pathlib import Path

In [2]:
df = pd.read_csv(Path('./data/20200906.csv').resolve())
df

,Timestamp,Rank your movie choices. You don't have to do a full ranking. [First Choice],Rank your movie choices. You don't have to do a full ranking. [Second Choice],Rank your movie choices. You don't have to do a full ranking. [Third Choice],Rank your movie choices. You don't have to do a full ranking. [Fourth Choice],Rank your movie choices. You don't have to do a full ranking. [Fifth Choice],Rank your movie choices. You don't have to do a full ranking. [Sixth Choice],Rank your movie choices. You don't have to do a full ranking. [Seventh Choice],What theme should we explore next week? [First Choice],What theme should we explore next week? [Second Choice],What theme should we explore next week? [Third Choice],What theme should we explore next week? [Fourth Choice],What theme should we explore next week? [Fifth Choice],What theme should we explore next week? [Sixth Choice],What theme should we explore next week? [Seventh Choice],What theme should we explore next week? [Eighth Choice]
0,9/1/2020 18:34:35,McCabe & Mrs. Miller,"Jeanne Dielman, 23 Commerce Quay, 1080 Bruxelles",Belladonna of Sadness,The Terminal Man,NaN,NaN,NaN,A GODDAMNED GHOST,LADY DIRECTOR,"POST-9/11, PRE-DEATH OF WHITNEY HOUSTON TYPE BEAT",PERIOD PIECES,BIPOC WRITERS/POETS,JOHN LEGUIZAMO,PRUPPETS,No Theme/Free-for-all
1,9/1/2020 18:36:30,"Aguirre, the Wrath of God",McCabe & Mrs. Miller,Badlands,Belladonna of Sadness,Get Carter,The Terminal Man,"Jeanne Dielman, 23 Commerce Quay, 1080 Bruxelles",JOHN LEGUIZAMO,BIPOC WRITERS/POETS,A GODDAMNED GHOST,LADY DIRECTOR,PERIOD PIECES,PRUPPETS,"POST-9/11, PRE-DEATH OF WHITNEY HOUSTON TYPE BEAT",No Theme/Free-for-all
2,9/1/2020 18:40:52,Badlands,"Aguirre, the Wrath of God",Get Carter,The Terminal Man,"Jeanne Dielman, 23 Commerce Quay, 1080 Bruxelles",McCabe & Mrs. Miller,Belladonna of Sadness,A GODDAMNED GHOST,BIPOC WRITERS/POETS,LADY DIRECTOR,JOHN LEGUIZAMO,PERIOD PIECES,"POST-9/11, PRE-DEATH OF WHITNEY HOUSTON TYPE BEAT",PRUPPETS,No Theme/Free-for-all
3,9/1/2020 19:05:47,Badlands,Belladonna of Sadness,Get Carter,"Aguirre, the Wrath of God","Jeanne Dielman, 23 Commerce Quay, 1080 Bruxelles",McCabe & Mrs. Miller,The Terminal Man,PRUPPETS,BIPOC WRITERS/POETS,A GODDAMNED GHOST,LADY DIRECTOR,JOHN LEGUIZAMO,PERIOD PIECES,"POST-9/11, PRE-DEATH OF WHITNEY HOUSTON TYPE BEAT",No Theme/Free-for-all
4,9/1/2020 20:36:00,"Aguirre, the Wrath of God",Badlands,The Terminal Man,Get Carter,"Jeanne Dielman, 23 Commerce Quay, 1080 Bruxelles",McCabe & Mrs. Miller,Belladonna of Sadness,PERIOD PIECES,A GODDAMNED GHOST,BIPOC WRITERS/POETS,LADY DIRECTOR,JOHN LEGUIZAMO,"POST-9/11, PRE-DEATH OF WHITNEY HOUSTON TYPE BEAT",PRUPPETS,No Theme/Free-for-all
5,9/2/2020 13:37:14,Badlands,Get Carter,"Aguirre, the Wrath of God",The Terminal Man,McCabe & Mrs. Miller,Belladonna of Sadness,"Jeanne Dielman, 23 Commerce Quay, 1080 Bruxelles",PRUPPETS,"POST-9/11, PRE-DEATH OF WHITNEY HOUSTON TYPE BEAT",LADY DIRECTOR,PERIOD PIECES,BIPOC WRITERS/POETS,A GODDAMNED GHOST,JOHN LEGUIZAMO,No Theme/Free-for-all
6,9/2/2020 16:00:38,Get Carter,Badlands,The Terminal Man,"Aguirre, the Wrath of God",McCabe & Mrs. Miller,NaN,NaN,"POST-9/11, PRE-DEATH OF WHITNEY HOUSTON TYPE BEAT",LADY DIRECTOR,PERIOD PIECES,BIPOC WRITERS/POETS,A GODDAMNED GHOST,PRUPPETS,JOHN LEGUIZAMO,NaN
7,9/2/2020 16:04:46,Get Carter,Badlands,"Jeanne Dielman, 23 Commerce Quay, 1080 Bruxelles","Aguirre, the Wrath of God",McCabe & Mrs. Miller,The Terminal Man,Belladonna of Sadness,LADY DIRECTOR,"POST-9/11, PRE-DEATH OF WHITNEY HOUSTON TYPE BEAT",BIPOC WRITERS/POETS,A GODDAMNED GHOST,PERIOD PIECES,PRUPPETS,JOHN LEGUIZAMO,No Theme/Free-for-all
8,9/2/2020 17:21:34,Belladonna of Sadness,"Aguirre, the Wrath of God",Badlands,"Jeanne Dielman, 23 Commerce Quay, 1080 Bruxelles",McCabe & Mrs. Miller,Get Carter,The Terminal Man,BIPOC WRITERS/POETS,JOHN LEGUIZAMO,"POST-9/11, PRE-DEATH OF WHITNEY HOUSTON TYPE BEAT",LADY DIRECTOR,PERIOD PIECES,PRUPPETS,A GODDAMNED GHOST,No Theme/Free-for-all
9,9/2/2020 19:28:21,"Jeanne Dielm

In [3]:
just_ballots = df.loc[:, df.columns.str.startswith('Rank')]
next_week = df.loc[:, df.columns.str.startswith('What')]

In [4]:
submissions = just_ballots.loc[just_ballots.notna().all(axis=1).idxmax(),:].values
submissions

array(['Aguirre, the Wrath of God', 'McCabe & Mrs. Miller', 'Badlands',
       'Belladonna of Sadness', 'Get Carter', 'The Terminal Man',
       'Jeanne Dielman, 23 Commerce Quay, 1080 Bruxelles'], dtype=object)

In [5]:
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}

In [6]:
balloted = just_ballots.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)

In [7]:
submission_dict

{'Aguirre, the Wrath of God': <Candidate('Aguirre, the Wrath of God')>,
 'McCabe & Mrs. Miller': <Candidate('McCabe & Mrs. Miller')>,
 'Badlands': <Candidate('Badlands')>,
 'Belladonna of Sadness': <Candidate('Belladonna of Sadness')>,
 'Get Carter': <Candidate('Get Carter')>,
 'The Terminal Man': <Candidate('The Terminal Man')>,
 'Jeanne Dielman, 23 Commerce Quay, 1080 Bruxelles': <Candidate('Jeanne Dielman, 23 Commerce Quay, 1080 Bruxelles')>}

In [8]:
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))

In [9]:
ballots

[<Ballot(McCabe & Mrs. Miller, Jeanne Dielman, 23 Commerce Quay, 1080 Bruxelles, Belladonna of Sadness, The Terminal Man)>,
 <Ballot(Aguirre, the Wrath of God, McCabe & Mrs. Miller, Badlands, Belladonna of Sadness, Get Carter, The Terminal Man, Jeanne Dielman, 23 Commerce Quay, 1080 Bruxelles)>,
 <Ballot(Badlands, Aguirre, the Wrath of God, Get Carter, The Terminal Man, Jeanne Dielman, 23 Commerce Quay, 1080 Bruxelles, McCabe & Mrs. Miller, Belladonna of Sadness)>,
 <Ballot(Badlands, Belladonna of Sadness, Get Carter, Aguirre, the Wrath of God, Jeanne Dielman, 23 Commerce Quay, 1080 Bruxelles, McCabe & Mrs. Miller, The Terminal Man)>,
 <Ballot(Aguirre, the Wrath of God, Badlands, The Terminal Man, Get Carter, Jeanne Dielman, 23 Commerce Quay, 1080 Bruxelles, McCabe & Mrs. Miller, Belladonna of Sadness)>,
 <Ballot(Badlands, Get Carter, Aguirre, the Wrath of God, The Terminal Man, McCabe & Mrs. Miller, Belladonna of Sadness, Jeanne Dielman, 23 Commerce Quay, 1080 Bruxelles)>,
 <Ballot(Ge

In [10]:
election_result = pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,)

In [11]:
print(election_result)

ROUND 1
Candidate                                           Votes  Status
------------------------------------------------  -------  --------
Badlands                                                3  Hopeful
Get Carter                                              2  Hopeful
Aguirre, the Wrath of God                               2  Hopeful
Jeanne Dielman, 23 Commerce Quay, 1080 Bruxelles        1  Hopeful
Belladonna of Sadness                                   1  Hopeful
McCabe & Mrs. Miller                                    1  Rejected
The Terminal Man                                        0  Rejected

ROUND 2
Candidate                                           Votes  Status
------------------------------------------------  -------  --------
Badlands                                                3  Hopeful
Get Carter                                              2  Hopeful
Aguirre, the Wrath of God                               2  Hopeful
Jeanne Dielman, 23 Commerce Quay, 1080 Brux

In [12]:
submissions = next_week.loc[next_week.notna().all(axis=1).idxmax(),:].values
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}
balloted = next_week.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))
print(pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,))

ROUND 1
Candidate                                            Votes  Status
-------------------------------------------------  -------  --------
LADY DIRECTOR                                            2  Hopeful
A GODDAMNED GHOST                                        2  Hopeful
PRUPPETS                                                 2  Hopeful
BIPOC WRITERS/POETS                                      1  Hopeful
POST-9/11, PRE-DEATH OF WHITNEY HOUSTON TYPE BEAT        1  Hopeful
PERIOD PIECES                                            1  Hopeful
JOHN LEGUIZAMO                                           1  Rejected
No Theme/Free-for-all                                    0  Rejected

ROUND 2
Candidate                                            Votes  Status
-------------------------------------------------  -------  --------
LADY DIRECTOR                                            2  Hopeful
BIPOC WRITERS/POETS                                      2  Hopeful
A GODDAMNED GHOST            